In [ ]:
import pandas as pd
import os
import numpy as np
from pathlib import Path
import math
import torch
from copy import deepcopy
from itertools import chain 
from torch.utils.data import Dataset
from torchtext.vocab import vocab as Vocab

import warnings
warnings.filterwarnings("ignore")


Pathing


In [1]:
#Lokalt
data_dir = 'c:\\Users\\erika\\Desktop\\Exjobb\\data'

In [ ]:
#saga
data_dir = "/home/aeerik/data/raw/"


Data import

In [ ]:
from data_preprocessing import data_loader
include_pheno = False
threshold_year = 1970
path = data_dir

NCBI = data_loader(include_pheno,threshold_year,path)
NCBI.head()

Vocabulary

In [ ]:
from build_vocabulary import make_vocabulary
vocabulary = make_vocabulary(NCBI)
print(len(vocabulary))

In [2]:
from create_dataset import NCBIDataset
from build_vocabulary import make_vocabulary
from data_preprocessing import data_loader

include_pheno = False
threshold_year = 1970
path = data_dir

NCBI = data_loader(include_pheno,threshold_year,path)

max_length = 20
mask_prob = 0.30
vocabulary = make_vocabulary(NCBI)

test_set = NCBIDataset(NCBI, vocabulary, max_length, mask_prob)
test_set.prepare_dataset()

Embedding 

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class JointEmbedding(nn.Module):

    def __init__(self, embedding_dim, vocab_size, max_length, drop_prob):
        super(JointEmbedding, self).__init__()

        self.max_length = max_length
        self.drop_prob = drop_prob
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim

        self.token_emb = nn.Embedding(self.vocab_size, self.embedding_dim)

        self.dropout = nn.Dropout(drop_prob)	
        self.norm = nn.LayerNorm(self.embedding_dim)

    def forward(self, input_tensor):
        token_embedding = self.token_emb(input_tensor)
        token_embedding = self.norm(token_embedding)
        token_embedding = self.dropout(token_embedding)

        return token_embedding


In [28]:
from embedding import JointEmbedding
embedding_dim = 32
voca_size = len(vocabulary)
max_length = 20
drop_prob = 0.2

emb_test = JointEmbedding(embedding_dim, voca_size, max_length, drop_prob)
emb_test.forward(test_set[1][0])

tensor([[ 1.3906, -0.0000,  1.5145, -0.0000,  1.7002,  1.1402, -0.0000, -0.9546,
          1.3977,  1.9938,  0.4370,  0.2733, -0.0000,  1.3827,  0.7178, -0.9676,
         -1.1515, -1.2852, -1.8969, -1.5310, -0.8330,  0.5568,  0.2334, -0.6911,
         -0.9162, -0.0750, -0.0000,  2.3868,  0.3402,  0.5523,  1.0168,  0.4211],
        [ 0.6452,  0.0000,  0.0568, -0.5182, -0.0000,  1.4908,  2.0379, -0.0504,
         -0.0000,  0.0000,  0.0790, -2.6247, -0.5794,  1.0984,  0.6177, -0.9772,
          1.2881,  0.1932, -0.1510, -0.3476,  1.4219,  0.7506, -0.0000, -0.0000,
         -1.6010,  1.2559, -0.4444, -3.2181, -0.3482,  1.6645,  0.0000,  0.8628],
        [ 1.9827,  0.2361,  1.7897, -2.7533, -0.0000,  0.9367, -0.2634,  0.0000,
         -1.1148,  0.0227,  1.5904,  1.7657,  0.4263, -1.4521,  1.1401, -1.3388,
          0.4797,  0.5975,  0.1636, -2.2488,  1.7180, -0.3915, -0.7171, -0.7517,
         -0.0000, -0.6126, -0.0949, -1.5568,  0.6963,  1.1347,  0.3446, -0.0000],
        [-1.4923,  0.0000

In [ ]:
import torch

from torch import nn
import torch.nn.functional as f

class AttentionHead(nn.Module):

    def __init__(self, dim_inp, dim_out, drop_prob):
        super(AttentionHead, self).__init__()

        self.dim_inp = dim_inp
        self.drop_prob = drop_prob
        self.dropout = nn.Dropout(drop_prob)
        self.q = nn.Linear(dim_inp, dim_out)
        self.k = nn.Linear(dim_inp, dim_out)
        self.v = nn.Linear(dim_inp, dim_out)

    def forward(self, input_tensor: torch.Tensor, attention_mask: torch.Tensor = None):
        query, key, value = self.q(input_tensor), self.k(input_tensor), self.v(input_tensor)

        scale = query.size(1) ** 0.5
        scores = torch.matmul(query, key.transpose(-1, -2)) / scale

        scores = scores.masked_fill_(attention_mask, -1e9)
        attn = f.softmax(scores, dim=-1)
        attn = self.dropout(attn)
        context = torch.matmul(attn, value)

        return context


class MultiHeadAttention(nn.Module):

    def __init__(self, num_heads, dim_inp, dim_out,drop_prob):
        super(MultiHeadAttention, self).__init__()

        self.heads = nn.ModuleList([
            AttentionHead(dim_inp, dim_out,drop_prob) for _ in range(num_heads)
        ])
        self.linear = nn.Linear(dim_out * num_heads, dim_inp)
        self.norm = nn.LayerNorm(dim_inp)

    def forward(self, input_tensor: torch.Tensor, attention_mask: torch.Tensor):
        s = [head(input_tensor, attention_mask) for head in self.heads]
        scores = torch.cat(s, dim=-1)
        scores = self.linear(scores)
        return self.norm(scores)
    
attention_test = AttentionHead(32, 32, 0.2)
attention_test.forward(emb_test.forward(test_set[1][0]), test_set[1][2])

multihead_test = MultiHeadAttention(8, 32, 32,0.2)
multihead_test.forward(emb_test.forward(test_set[1][0]), test_set[1][2])

In [ ]:
import torch

from torch import nn
import torch.nn.functional as f

#dropout 
#num heads
#dim in, dim out

class Encoder(nn.Module):

    def __init__(self, dim_inp, dim_out, attention_heads, dropout_prob):
        super(Encoder, self).__init__()
        self.dropout_prob = dropout_prob
        self.attention_heads = attention_heads
        self.dim_inp = dim_inp
        self.dim_out = dim_out
        
        
        self.attention = MultiHeadAttention(self.attention_heads, self.dim_inp, self.dim_out, self.dropout_prob)  # batch_size x sentence size x dim_inp
        self.feed_forward = nn.Sequential(
            nn.Linear(self.dim_inp, self.dim_out),
            nn.Dropout(self.dropout_prob),
            nn.GELU(),
            nn.Linear(self.dim_out, self.dim_inp),
            nn.Dropout(self.dropout_prob)
        )
        self.norm = nn.LayerNorm(self.dim_inp)

    def forward(self, input_tensor: torch.Tensor, attention_mask: torch.Tensor):
        context = self.attention(input_tensor, attention_mask)
        res = self.feed_forward(context)
        return self.norm(res)

encoder_test = Encoder(32, 32, 8, 0.2)
encoder_test.forward(emb_test.forward(test_set[1][0]), test_set[1][2])

In [25]:
class BERT(nn.Module):

    def __init__(self, vocab_size, max_length, dim_inp, dim_out, attention_heads, num_encoders, dropout_prob):
        super(BERT, self).__init__()
        self.attention_heads = attention_heads
        self.max_length = max_length
        self.vocab_size = vocab_size
        self.dim_inp = dim_inp  
        self.dim_out = dim_out
        self.num_encoders = num_encoders
        self.dropout_prob = dropout_prob  

        self.embedding = JointEmbedding(self.dim_inp, self.vocab_size, self.max_length, self.dropout_prob)
        self.encoders = nn.ModuleList([Encoder(self.dim_inp, self.dim_out, self.attention_heads, self.dropout_prob) for _ in range(self.num_encoders)])

        self.token_prediction_layer = nn.Linear(self.dim_inp, self.vocab_size)
        self.softmax = nn.LogSoftmax(dim=-1)

    def forward(self, input_tensor: torch.Tensor, attention_mask: torch.Tensor):
        embedded = self.embedding(input_tensor)
        for layer in self.encoders:
            embedded = layer(embedded, attention_mask)

        token_predictions = self.token_prediction_layer(embedded)
        return self.softmax(token_predictions)

In [26]:
bert_test = BERT(len(vocabulary), 20, 32, 32, 8, 2, 0.2)
bert_test.forward(test_set[1][0], test_set[1][2])

NameError: name 'Encoder' is not defined

In [29]:
from bert_builder import AttentionHead

attention_test = AttentionHead(32, 32, 0.2)
attention_test.forward(emb_test.forward(test_set[1][0]), test_set[1][2])

tensor([[-2.0241e-02,  1.5273e-01,  2.6295e-01,  6.6555e-01, -1.0036e-01,
         -6.5821e-01, -1.2605e-02,  7.7466e-01,  4.7497e-01, -8.1123e-01,
         -5.8339e-01, -6.4419e-01, -1.4982e-01,  2.8228e-01,  6.2414e-01,
          5.8298e-01, -7.0692e-01,  1.1555e-03, -4.7954e-01, -1.7100e-01,
          3.6572e-01,  5.3296e-02, -1.7360e-01,  3.0050e-01, -9.3018e-01,
         -1.6014e-01,  2.2848e-02, -3.6400e-02,  6.6105e-01, -4.2749e-01,
         -5.0100e-01,  1.4583e-01],
        [-2.6248e-01, -1.3555e-01,  1.8824e-01,  6.2918e-01, -3.6234e-03,
         -4.0952e-01, -1.8347e-01,  7.8743e-01,  5.0052e-01, -6.9197e-01,
         -6.6147e-01, -6.2018e-01, -5.7642e-02,  2.5905e-01,  8.6495e-01,
          5.9880e-01, -4.1036e-01,  9.8861e-02, -3.3702e-01, -5.4992e-02,
          3.4315e-01, -1.9010e-01, -1.3948e-01,  2.2546e-01, -7.8482e-01,
         -9.8081e-02,  7.9229e-02,  2.0034e-01,  7.7362e-01, -5.6719e-01,
         -4.7929e-01,  3.8551e-01],
        [-4.7225e-01, -2.1066e-01,  2.09

In [30]:
from bert_builder import MultiHeadAttention
multihead_test = MultiHeadAttention(num_heads=8, dim_inp=32, dim_out=32, drop_prob=0.2)
multihead_test.forward(emb_test.forward(test_set[1][0]), test_set[1][2])

tensor([[ 1.4130, -0.0907,  0.1716, -0.5289, -0.8205, -1.7303,  1.2896, -0.5984,
         -0.5620,  1.1284,  1.3726,  0.1151, -1.5881, -0.3556,  0.2278, -1.8129,
         -0.3096,  0.9635, -0.1761, -0.8337,  1.0715,  1.9798, -0.8971,  0.8563,
         -0.7656, -0.8133, -0.7476, -0.6008, -0.4917,  0.5151,  1.1399,  1.4786],
        [ 1.3435, -0.9398, -0.1196, -1.3160, -0.5235, -1.0834, -0.4125, -0.2887,
         -0.4767,  1.0614,  0.8628,  0.8140, -1.0553, -0.7731,  0.7286,  0.1749,
         -0.4641,  2.0798, -0.4040, -1.3819,  0.6538,  1.6230,  0.1166,  1.1454,
         -0.7040, -1.6622, -0.8966, -1.3125,  0.7005,  0.8835, -0.0376,  1.6638],
        [ 1.3853, -1.0699, -0.0670, -1.0698, -0.4439, -1.5180,  0.2586, -0.1898,
         -0.0194,  1.1598,  0.6623,  0.1378, -1.0881, -0.3579,  0.3343, -0.0998,
         -0.2816,  2.0297, -0.1493, -1.2266,  1.1505,  1.4389, -0.1601,  1.1152,
         -0.5965, -2.1915, -0.9558, -0.9647,  0.0598,  1.2299, -0.2373,  1.7252],
        [ 1.4894, -1.1520

In [31]:
from bert_builder import Encoder

encoder_test = Encoder(32, 32, 8, 0.2)
encoder_test.forward(emb_test.forward(test_set[1][0]), test_set[1][2])

tensor([[ 2.0248e-01,  2.0311e+00, -1.6133e+00,  7.2024e-01, -1.7753e+00,
         -3.3829e-01, -2.0464e+00,  5.9467e-01, -1.1393e-01, -2.7632e-01,
          1.5230e+00,  1.0057e+00, -1.0793e+00, -1.7486e-01, -4.3740e-01,
         -8.5663e-01,  2.2595e-01,  1.3982e-01, -5.1596e-03,  8.6017e-01,
          1.1884e+00,  1.0518e-01,  1.3982e-01,  1.3982e-01,  1.3982e-01,
         -1.7964e+00,  1.0775e+00, -1.7099e+00,  2.0946e-01,  1.5297e-01,
          6.2984e-01,  1.1371e+00],
        [ 1.3799e-01,  2.1013e+00, -1.2263e-01,  7.0940e-01, -2.5010e+00,
          7.5570e-01, -9.2116e-01,  6.5230e-01, -5.8747e-01, -3.7981e-01,
         -1.8622e-01,  4.5554e-02,  1.3799e-01, -1.6211e-01,  1.3799e-01,
         -1.2445e+00, -1.1914e+00,  1.3799e-01,  1.0154e-01,  8.0911e-01,
          1.3799e-01,  1.1312e+00,  1.1770e+00,  9.9842e-01,  1.3799e-01,
         -1.5109e+00,  9.8507e-02, -2.0704e+00, -2.8472e-01, -7.0512e-01,
          5.8916e-01,  1.8703e+00],
        [ 8.4869e-01,  5.9055e-01, -1.16

In [32]:
from bert_builder import BERT

bert_test = BERT(vocab_size=len(vocabulary), max_length=20, dim_inp=32, dim_out=32, attention_heads=8, num_encoders=2, dropout_prob=0.2)
bert_test.forward(test_set[1][0], test_set[1][2])

tensor([[-6.6870, -8.2247, -8.0846,  ..., -7.4633, -7.4026, -7.4531],
        [-7.1619, -7.7226, -8.5259,  ..., -8.0916, -7.3004, -7.0482],
        [-6.8304, -7.1615, -8.2725,  ..., -8.0485, -6.7234, -6.3459],
        ...,
        [-6.9619, -7.4826, -8.2713,  ..., -7.4195, -6.7692, -7.4150],
        [-7.1781, -7.1967, -8.1278,  ..., -7.6795, -6.7403, -6.7293],
        [-6.9291, -7.2373, -8.1712,  ..., -7.1587, -6.9085, -7.2713]],
       grad_fn=<LogSoftmaxBackward0>)

---------------------------

Trainer

In [89]:
import numpy as np
from create_dataset import NCBIDataset
def get_split_indices(size_to_split, val_share, random_state: int = 42):
    indices = np.arange(size_to_split)
    np.random.seed(random_state)
    np.random.shuffle(indices)
    
    train_share = 1 - val_share
    
    train_size = int(train_share * size_to_split)
    
    train_indices = indices[:train_size]
    val_indices = indices[train_size:]
    
    return train_indices, val_indices
max_length = 20
mask_prob = 0.30
vocabulary = make_vocabulary(NCBI)

train_indices, val_indices = get_split_indices(len(NCBI), 0.2)
train_set = NCBIDataset(NCBI.iloc[train_indices], vocabulary, max_length, mask_prob)
val_set = NCBIDataset(NCBI.iloc[val_indices], vocabulary, max_length, mask_prob)

In [91]:
train_set.prepare_dataset() 


KeyError: 'genes'

In [92]:
from create_dataset import NCBIDataset
test_set = NCBIDataset(NCBI, vocabulary, max_length, mask_prob)


In [94]:
test_set.prepare_dataset()

KeyError: 'genes'

In [40]:
import time
from datetime import datetime
from torch import nn
from torch.utils.data import DataLoader


class BertTrainer:
    def __init__(self, model, train_set, val_set, epochs, batch_size, lr, device, results_dir):
        
        self.model = model
        self.train_set = train_set
        self.val_set = val_set
        self.epochs = epochs    
        self.batch_size = batch_size
        self.lr = lr
        self.weight_decay = 0.01
        self.current_epoch  = 0

        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr, weight_decay=self.weight_decay)
        self.criterion = nn.CrossEntropyLoss(ignore_index = -1).to(device)

        self.device = device
        self.results_dir = results_dir


    def __call__(self):      
        self.val_set.prepare_dataset() 
        self.val_loader = DataLoader(self.val_set, batch_size=self.batch_size, shuffle=False)
        
        start_time = time.time()
        self.best_val_loss = float('inf')
        self._init_result_lists()
        for self.current_epoch in range(self.current_epoch, self.epochs):
            self.model.train()
            self.train_set.prepare_dataset()
            self.train_loader = DataLoader(self.train_set, batch_size=self.batch_size, shuffle=True)
            epoch_start_time = time.time()
            avg_epoch_loss = self.train(self.current_epoch)


    def _init_result_lists(self):
        self.losses = []
        self.val_losses = []
        self.val_accs = []
        self.val_iso_accs = []
        self.val_iso_stats = []

    def train(self, epoch: int):
        print(f"Epoch {epoch+1}/{self.epochs}")
        time_ref = time.time()
        
        epoch_loss = 0
        reporting_loss = 0
        printing_loss = 0
        for i, batch in enumerate(self.train_loader):
            print(f"starting")
            input, token_target, attn_mask = batch
            
            self.optimizer.zero_grad() 
            tokens = self.model(input, attn_mask) 
            print(f"tokens: {tokens.shape}")
            print(f"tokens_target: {token_target.shape}")
            loss = self.criterion(tokens.transpose(-1, -2), token_target) 
            
            epoch_loss += loss.item() 
            reporting_loss += loss.item()
            printing_loss += loss.item()
            
            loss.backward() 
            self.optimizer.step()         
        avg_epoch_loss = epoch_loss / self.num_batches
        return avg_epoch_loss 

In [99]:
max_length = 51
mask_prob = 0.30
vocabulary = make_vocabulary(NCBI)

train_indices, val_indices = get_split_indices(len(NCBI), 0.2)
train_set = NCBIDataset(NCBI.iloc[train_indices], vocabulary, max_length, mask_prob)
val_set = NCBIDataset(NCBI.iloc[val_indices], vocabulary, max_length, mask_prob)

test = BertTrainer(bert_test, train_set, val_set, 1, 32, 0.001, device, "results_dir")
results = test()

Epoch 1/1


RuntimeError: stack expects each tensor to be equal size, but got [20] at entry 0 and [21] at entry 12